You can refer [here](https://blog.rasa.com/rasa-nlu-in-depth-part-1-intent-classification/) for the inception of this idea. 

In [1]:
from sklearn import svm
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
import numpy as np
import os
from os import sys
from os import path
sys.path.append(path.dirname(path.dirname(os.getcwd())))

In [2]:
svm.SVC()

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [3]:
from speech.utils import db

In [4]:
signals, colnames = db.execute_query('select * from product_signal where product_id = 390')

/home/absin/git/ai/speech/utils/constants.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  doc = yaml.load(f)


In [8]:
X_raw = []
Y_raw = []
for signal in signals:
    if signal[colnames.index('is_active')]:
        X_raw.append(signal[colnames.index('value')])
        Y_raw.append(signal[colnames.index('name')])
print(X_raw)
print(Y_raw)

['usd', 'payment link', 'rupees', 'US dollars', 'how much', 'Ethical Hacking Methodology', 'penetration testing', 'American National Standards Institute', 'ANSI', 'CNSS', 'Cloud Computing', 'SQL Injection', 'Hacking Mobile Platforms', 'Hacking Wireless Networks', 'Live Instructor led training', 'EC-Council', 'security analyst', 'forensic investigation', 'Sell the shade, not the tree', 'Hello', 'HI', 'Hello', 'Thanks', 'exam', 'Our next class', 'Feature', 'iLearn', 'iLabs', 'classroom training', 'ethical hacking', 'Exam Voucher', 'CEH v9 Certification', 'CEH V9 training', 'Security Professional', 'CEH V9', 'Certified Ethical Hacker Certification', 'technical certification', 'Ecourseware', 'certified ethical hacker', 'IWeek', 'EC Council', 'complete training', 'ecse program', 'train professors', 'digital book', 'online training']
['Pricing', 'Pricing', 'Pricing', 'Pricing', 'Discount', 'Value Prop Signal', 'Value Prop Signal', 'Value Prop Signal', 'Value Prop Signal', 'Value Prop Signal'

In [10]:
import tensorflow_hub as hub
import tensorflow as tf
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(module_url)
g = tf.get_default_graph()
session = tf.Session(graph=g)
session.run([tf.global_variables_initializer(), tf.tables_initializer()])
messages = tf.placeholder(dtype=tf.string, shape=[None])
output = embed(messages)
print('tf encoder ready')

tf encoder ready


In [26]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(Y_raw)
x_enc = []
x = []
with g.as_default():
    x_enc = list(session.run(output, feed_dict={messages:  X_raw}))
for i, x1 in enumerate(x_enc):
    x.append(list(x1)) 
print(y)
print(x)

[6 6 6 6 1 8 8 8 8 8 8 8 8 8 8 0 5 5 4 3 3 3 3 8 2 2 7 8 2 2 1 7 7 7 7 7 7
 7 7 7 0 2 2 2 2 2]
[[-0.03674993, -0.047180057, -0.032831986, -0.03968859, 0.022259435, -0.06904287, -0.033820983, 0.043528024, -0.053894166, 0.067606814, -0.002389255, -0.013478262, 0.03284705, 0.07597626, 0.061670683, -0.034513067, -0.0522905, 0.04960871, 0.07743611, -0.04445091, -0.01963947, 0.036826003, -0.037076015, 0.08498421, 0.026805785, -0.07397137, -0.020790637, 0.026268441, 0.031706512, 0.02321557, -0.02063283, -0.0051853782, 0.018460408, 0.04025412, -0.024179656, -0.0019727394, 0.007900006, 0.0510664, -0.01302708, 0.009187068, 0.029296631, 0.012114173, -0.05742694, -0.027908415, -0.011830864, 0.034087136, 0.038205493, 0.03515535, 0.04214587, 0.045935936, 0.0753062, -0.039538786, 0.014712934, 0.033164755, -0.08889908, -0.068568386, -0.029276041, 0.029191772, -0.04985015, -0.06773976, -0.051766437, -0.035094734, 0.0040429295, 0.035751835, 0.06666664, -0.07036904, 0.012274277, -0.030448189, 0.06489147,

In [29]:
np.inner(x[19],x[1])

0.2819887

In [30]:
defaults = {
    "C": [1, 2, 5, 10, 20, 100],
    "kernels": ["linear"],
    "max_cross_validation_folds": 5}
C = defaults["C"]
kernels = defaults["kernels"]
tuned_parameters = [{"C": C, "kernel": [str(k) for k in kernels]}]
folds = defaults["max_cross_validation_folds"]
cv_splits = max(2, min(folds, np.min(np.bincount(y)) // 5))

In [31]:
clf = GridSearchCV(svm.SVC(C=1, probability=True, class_weight='balanced'),
                   param_grid=tuned_parameters,
                   n_jobs=1,
                   cv=cv_splits,         
                   scoring='f1_weighted', 
                   verbose=1)

In [33]:
clf.fit(x, y)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


/home/absin/git/ai/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/absin/git/ai/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/absin/git/ai/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/absin/git/ai/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and b

GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=SVC(C=1, cache_size=200, class_weight='balanced',
                           coef0=0.0, decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=1,
             param_grid=[{'C': [1, 2, 5, 10, 20, 100], 'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_weighted', verbose=1)

In [39]:
with g.as_default():
    x_pred = list(session.run(output, feed_dict={messages:  ["how did you apply"]}))
    print(le.inverse_transform(clf.predict(list(x_pred))))

['Discount']


In [104]:
y

array([6, 1, 8, 0, 5, 4, 3, 2, 7])

In [105]:
ZZX[0]

[0, 0]